This notebook loads our raw dataset and processes it to create a processed dataset for clustering (no labels) or classification (bearish/bullish label) algorithms.

While exploring and processing the raw dataset, the following was considered:

    - Scaling/standardization: variables had very different scales requiring standardization
    - Class imbalance: certain crypto currencies have significantly more datapoints available than others
    - Nulls: none of the rows have nulls but certain financial technical indicators are averaged over a timespan, leading to some nulls that needed to be removed/dropped.
    - Recoding variables: Categorical variables were numerically encoded
    - Data leakage: Given that we are dealing with time series data. Care was taken to split data by time instead of randomly. In addition, data was split between train and test set prior to scaling.
    - Symbol is not a unique identifer for crypto currencies; slug/name is unique identifer. E.g. Webchain and webcoin both share the same symbol WEB and have different pricing information.

In [2]:
import pandas as pd

#Import given crypto dataset into pandas dataframe
df_raw = pd.read_csv('crypto-markets.csv')
display(df_raw.head())

,slug,symbol,name,date,ranknow,open,high,low,close,volume,market,close_ratio,spread
0,bitcoin,BTC,Bitcoin,2013-04-28,1,135.30,135.98,132.10,134.21,0.0,1.488567e+09,0.5438,3.88
1,bitcoin,BTC,Bitcoin,2013-04-29,1,134.44,147.49,134.00,144.54,0.0,1.603769e+09,0.7813,13.49
2,bitcoin,BTC,Bitcoin,2013-04-30,1,144.00,146.93,134.05,139.00,0.0,1.542813e+09,0.3843,12.88
3,bitcoin,BTC,Bitcoin,2013-05-01,1,139.00,139.89,107.72,116.99,0.0,1.298955e+09,0.2882,32.17
4,bitcoin,BTC,Bitcoin,2013-05-02,1,116.38,125.60,92.28,105.21,0.0,1.168517e+09,0.3881,33.32


In [3]:
# Convert 'date' column to datetime format
df_raw['date'] = pd.to_datetime(df_raw['date'], format = "%Y-%m-%d")
df_raw = df_raw.sort_values(by=['name', 'date'])
display(df_raw.head())

,slug,symbol,name,date,ranknow,open,high,low,close,volume,market,close_ratio,spread
224553,0chain,ZCN,0chain,2018-07-02,441,0.496757,0.530165,0.474749,0.479175,296945.0,0.0,0.0799,0.06
224554,0chain,ZCN,0chain,2018-07-03,441,0.480519,0.577536,0.467949,0.506107,290003.0,20244280.0,0.3482,0.11
224555,0chain,ZCN,0chain,2018-07-04,441,0.506340,0.537679,0.470698,0.505993,384747.0,20239720.0,0.5269,0.07
224556,0chain,ZCN,0chain,2018-07-05,441,0.507758,0.620443,0.506720,0.618253,290748.0,24730120.0,0.9807,0.11
224557,0chain,ZCN,0chain,2018-07-06,441,0.619071,0.619081,0.506407,0.566018,478795.0,22640720.0,0.5291,0.11


In [4]:
#Checking for any nulls in raw dataset
print(df_raw.isnull().sum())

slug           0
symbol         0
name           0
date           0
ranknow        0
open           0
high           0
low            0
close          0
volume         0
market         0
close_ratio    0
spread         0
dtype: int64


In [5]:
#Check if any of the pricing or volume variables are 0 in which case they should be dropped as likely missing or invalid
columns_to_check = ['open', 'high', 'low', 'close', 'volume', 'market']

for col in columns_to_check:
    zero_count = (df_raw[col] == 0).sum()
    print(f"{col}: {zero_count} zeros")
df_raw.info()

df_raw = df_raw[(df_raw[columns_to_check] != 0).all(axis=1)]

open: 0 zeros
high: 0 zeros
low: 0 zeros
close: 0 zeros
volume: 27820 zeros
market: 117697 zeros
<class 'pandas.core.frame.DataFrame'>
Index: 942297 entries, 224553 to 938202
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   slug         942297 non-null  object        
 1   symbol       942297 non-null  object        
 2   name         942297 non-null  object        
 3   date         942297 non-null  datetime64[ns]
 4   ranknow      942297 non-null  int64         
 5   open         942297 non-null  float64       
 6   high         942297 non-null  float64       
 7   low          942297 non-null  float64       
 8   close        942297 non-null  float64       
 9   volume       942297 non-null  float64       
 10  market       942297 non-null  float64       
 11  close_ratio  942297 non-null  float64       
 12  spread       942297 non-null  float64       
dtypes: datetime64[ns](1), float64(8), int

In [6]:
for col in columns_to_check:
    zero_count = (df_raw[col] == 0).sum()
    print(f"{col}: {zero_count} zeros")
df_raw.info()

open: 0 zeros
high: 0 zeros
low: 0 zeros
close: 0 zeros
volume: 0 zeros
market: 0 zeros
<class 'pandas.core.frame.DataFrame'>
Index: 800922 entries, 224554 to 695887
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   slug         800922 non-null  object        
 1   symbol       800922 non-null  object        
 2   name         800922 non-null  object        
 3   date         800922 non-null  datetime64[ns]
 4   ranknow      800922 non-null  int64         
 5   open         800922 non-null  float64       
 6   high         800922 non-null  float64       
 7   low          800922 non-null  float64       
 8   close        800922 non-null  float64       
 9   volume       800922 non-null  float64       
 10  market       800922 non-null  float64       
 11  close_ratio  800922 non-null  float64       
 12  spread       800922 non-null  float64       
dtypes: datetime64[ns](1), float64(8), int64(1), ob

In [7]:
#add additional technical indicators common for financial products to attempt to gain further possible insights. These are based on trend, momentum, volatility and volume dynamics
import pandas_ta as ta

df = df_raw.copy()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['name', 'date'])

#function that calculates indicators by crypto coin
def compute_indicators(group):
    group = group.copy()

    # Basic engineered features
    group['daily_return'] = (group['close'] - group['open']) / group['open']
    group['volume_change'] = group['volume'].pct_change()

    # Technical indicators
    group['sma_7'] = ta.sma(group['close'], length=7) #Simple Moving Average over 7 periods
    group['ema_14'] = ta.ema(group['close'], length=14) #Exponential Moving Average over 14 periods
    group['rsi_14'] = ta.rsi(group['close'], length=14) #Relative Strength Index calculated over 14 periods
    group['atr_14'] = ta.atr(group['high'], group['low'], group['close'], length=14) #Average True Range over 14 periods

    return group

#Calculate indicators for each crypto name
df_indicators = df.groupby('name', group_keys=False).apply(compute_indicators)

#Drop rows with missing indicator values which will occur for indicators using 7 or 14 day lag
indicator_cols = ['daily_return', 'volume_change', 'sma_7', 'ema_14', 'rsi_14', 'atr_14']

df_indicators = df_indicators.dropna(subset=indicator_cols)

#Reset index
df_indicators = df_indicators.reset_index(drop=True)

C:\Users\Descus\AppData\Local\Temp\ipykernel_15936\3756617224.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_indicators = df.groupby('name', group_keys=False).apply(compute_indicators)
C:\Users\Descus\AppData\Local\Temp\ipykernel_15936\3756617224.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_indicators = df.groupby('name', group_keys=False).apply(compute_indicators)


In [8]:
#Add bullish and bearish label based on price movement for possible further analysis
df_indicators['label'] = (df_indicators['close'] > df_indicators['open']).astype(int)
df_indicators.head()

,slug,symbol,name,date,ranknow,open,high,low,close,volume,market,close_ratio,spread,daily_return,volume_change,sma_7,ema_14,rsi_14,atr_14,label
0,0chain,ZCN,0chain,2018-07-17,441,0.426759,0.529069,0.421942,0.497478,121127.0,19899120.0,0.7051,0.11,0.165712,0.446552,0.449708,0.506991,49.929569,0.066502,1
1,0chain,ZCN,0chain,2018-07-18,441,0.497499,0.518483,0.457703,0.486524,62921.0,19460960.0,0.4742,0.06,-0.022060,-0.480537,0.454008,0.504262,48.334177,0.065893,0
2,0chain,ZCN,0chain,2018-07-19,441,0.486397,0.486397,0.439271,0.440240,87071.0,17609600.0,0.0206,0.05,-0.094896,0.383815,0.458029,0.495726,42.198661,0.063976,0
3,0chain,ZCN,0chain,2018-07-20,441,0.440344,0.440344,0.388898,0.396128,135686.0,15845120.0,0.1405,0.05,-0.100412,0.558337,0.451438,0.482446,37.334404,0.062726,0
4,0chain,ZCN,0chain,2018-07-21,441,0.396202,0.399500,0.374099,0.379937,61702.0,15197480.0,0.2298,0.03,-0.041052,-0.545259,0.440131,0.468778,35.707439,0.059107,0


In [9]:
#export processed raw data frame to a .csv
df_indicators.to_csv("crypto-markets-processed.csv", index=False)